In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
from pathlib import Path
import sys
import geopandas as gpd
import pandas as pd


scripts_dir = Path("../..").joinpath("src")
if scripts_dir not in sys.path:
    sys.path.insert(0, scripts_dir.resolve().as_posix())
from helpers.mapshaper import Mapshaper
from helpers.tippcanoe import mbtileGeneration
from helpers.mapbox_uploader import uploadToMapbox
from helpers.settings import get_settings
from helpers.file_handler import FileConventionHandler
from helpers.utils import download_and_unzip_if_needed, writeReadGCP

from data_commons.loader import load_regions

In [21]:
mysettings = get_settings()
prev_step = "preprocess"
current_step = "tiles"

### EEZs: Exclusive Economic Zones 

In [89]:
pipe = "eez"
collection_name = f"{pipe}_v11"

eez_dir = FileConventionHandler(pipe)

# Download the EEZ file && unzip it
download_and_unzip_if_needed(eez_dir, prev_step, mysettings)

/home/mambauser/data/eez/processed/eez_preprocess.zip
/home/mambauser/data/eez/processed/preprocess


In [90]:
# Download the translations file
input_file = eez_dir.get_processed_step_path(prev_step).joinpath("eez_translation.csv")

writeReadGCP(
    credentials=mysettings.GCS_KEYFILE_JSON,
    bucket_name=mysettings.GCS_BUCKET,
    blob_name="vizzuality_processed_data/eez/preprocess/eez_translation.csv",
    file=input_file,
    operation="r",
)

In [91]:
# Add translations
eez = gpd.read_file(eez_dir.get_step_fmt_file_path(prev_step, "shp").as_posix())
translations = pd.read_csv(input_file).drop(columns=["GEONAME", "POL_TYPE"])
eez_translation = eez.merge(translations, on="MRGID", how="left")
eez_translation.to_file(eez_dir.get_processed_step_path(prev_step).joinpath("eez_translation.shp").as_posix())

/opt/conda/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Value 212881389 of field AREA_KM2 of feature 281 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(


In [93]:
# Simplify the geometries
Mapshaper(8).input([eez_dir.get_processed_step_path(prev_step).joinpath("eez_translation.shp").as_posix()]).filter_fields(
    fields=",".join(["GEONAME", "GEONAME_ES", "GEONAME_FR", "POL_TYPE", "ISO_SOV1", "ISO_SOV2", "ISO_SOV3"])
).output(
    eez_dir.get_step_fmt_file_path(current_step, "json").as_posix(), force=True, format="geojson"
).execute()

# Generate mbtiles
mbtileGeneration(eez_dir.get_step_fmt_file_path(current_step, "json"))

For layer 0, using name "eez_tiles"
/home/mambauser/data/eez/processed/tiles/eez_tiles.json:10: Found ] at top level
/home/mambauser/data/eez/processed/tiles/eez_tiles.json:55: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","features":[]}
282 features, 62937109 bytes of geometry, 7882 bytes of separate metadata, 38990 bytes of string pool
Choosing a maxzoom of -z0 for features about 2511227 feet (765422 meters) apart
Choosing a maxzoom of -z9 for resolution of about 825 feet (251 meters) within features
  99.9%  9/136/131  


PosixPath('/home/mambauser/data/eez/processed/tiles/eez_tiles.mbtiles')

In [ ]:
uploadToMapbox(
    eez_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.mbtiles"),
    collection_name,
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

#### EEZs: wdpa Regions

In [29]:
collection_name = "regions"

# load the EEZ file & the regions file
eez_data = gpd.read_file(eez_dir.get_step_fmt_file_path(prev_step, "shp").as_posix())
regions_df = pd.DataFrame(
    [
        {"region_id": data["region_iso"], "location_id": iso}
        for data in load_regions().get("data", [])
        for iso in data["country_iso_3s"]
    ]
)

# merge the two files
gpd.GeoDataFrame(
    pd.merge(
        eez_data,
        regions_df,
        how="left",
        left_on="ISO_SOV1",
        right_on="location_id",
        sort=True,
        copy=True,
    ),
    crs=eez_data.crs,
).to_file(
    filename=eez_dir.get_processed_step_path(prev_step)
    .joinpath(f"{pipe}_{prev_step}_{collection_name}.shp")
    .as_posix(),
    driver="ESRI Shapefile",
)

# dissolve by region_id

Mapshaper(16).input(
    [
        eez_dir.get_processed_step_path(prev_step)
        .joinpath(f"{pipe}_{prev_step}_{collection_name}.shp")
        .as_posix()
    ]
).dissolve2(fields="region_id").output(
    eez_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.json").as_posix(),
    force=True,
    format="geojson",
).execute()

# generate the mbtiles
mbtileGeneration(eez_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.json"))

In [34]:
uploadToMapbox(
    eez_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.mbtiles"),
    collection_name,
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

upload: ../../data/eez/processed/tiles/regions.mbtiles to s3://tilestream-tilesets-production/97/_pending/ojc7oxn5cpu10yo0o9tsl1xlc/skytruth


Linking tileset to Mapbox: 100%|██████████| 100/100 [03:00<00:00,  1.81s/it]


True

### MPAs: Marine Protected Areas from WDPA

In [61]:
pipe = "mpa"
collection_name = "mpas_wdpa"

source_dir = FileConventionHandler(pipe)

# Download the EEZ file && unzip it
download_and_unzip_if_needed(source_dir, prev_step, mysettings)

# simplify the geometries
Mapshaper(16).input([source_dir.get_step_fmt_file_path(prev_step, "shp").as_posix()]).filter_fields(
    fields="WDPAID,NAME,PA_DEF,GIS_M_AREA,PARENT_ISO"
).clean(allow_overlaps=True, rewind=True).simplify("dp 10% keep-shapes planar").clean(
    allow_overlaps=True
).output(
    source_dir.get_step_fmt_file_path(current_step, "json").as_posix(), force=True, format="geojson"
).execute()

# generate the mbtiles
mbtileGeneration(source_dir.get_step_fmt_file_path(current_step, "json"))

/home/mambauser/data/mpa/processed/mpa_preprocess.zip
/home/mambauser/data/mpa/processed/preprocess


KeyboardInterrupt: 

In [ ]:
uploadToMapbox(
    source_dir.get_step_fmt_file_path(current_step, "mbtiles"),
    collection_name,
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

upload: ../../data/mpa/processed/tiles/mpa_tiles.mbtiles to s3://tilestream-tilesets-production/de/_pending/yvng0dxxxru12eq9ye80350mc/skytruth


Linking tileset to Mapbox: 100%|██████████| 100/100 [02:34<00:00,  1.54s/it]


True

### MPA Atlas 

In [7]:
pipe = "mpaatlas"
collection_name = "mpa_atlas"

source_dir = FileConventionHandler(pipe)
# Download the data file && unzip it if needed
download_and_unzip_if_needed(source_dir, prev_step, mysettings)

# generate the mbtiles
mbtileGeneration(
    source_dir.get_step_fmt_file_path(prev_step, "shp"),
    source_dir.get_step_fmt_file_path(current_step, "mbtiles"),
)

/home/mambauser/data/mpaatlas/processed/mpaatlas_preprocess.zip
/home/mambauser/data/mpaatlas/processed/preprocess


Allocating 50 GB of heap memory
[clean] Retained 863 of 870 features
[o] Wrote /home/mambauser/data/mpaatlas/processed/preprocess/mpaatlas_preprocess.json
For layer 0, using name "mpaatlas_preprocess"
/home/mambauser/data/mpaatlas/processed/preprocess/mpaatlas_preprocess.json:293: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","features":[]}
/home/mambauser/data/mpaatlas/processed/preprocess/mpaatlas_preprocess.json:21: Found ] at top level
863 features, 33449716 bytes of geometry, 29354 bytes of separate metadata, 53170 bytes of string pool
Choosing a maxzoom of -z0 for features about 282844 feet (86211 meters) apart
Choosing a maxzoom of -z12 for resolution of about 98 feet (30 meters) within features
  99.9%  12/1004/2052  


PosixPath('/home/mambauser/data/mpaatlas/processed/tiles/mpaatlas_tiles.mbtiles')

In [8]:
uploadToMapbox(
    source_dir.get_step_fmt_file_path(current_step, "mbtiles"),
    collection_name,
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

upload: ../../data/mpaatlas/processed/tiles/mpaatlas_tiles.mbtiles to s3://tilestream-tilesets-production/32/_pending/4py3xz71znm1h1p9cr1v050mc/skytruth


Linking tileset to Mapbox: 100%|██████████| 100/100 [05:24<00:00,  3.25s/it]


True

### Protected seas

In [37]:
pipe = "protectedseas"
collection_name = "protected_seas"

pipe_dir = FileConventionHandler(pipe)
input_file = pipe_dir.get_processed_step_path(prev_step).joinpath("ProtectedSeas_20240716.geojson")
output_file = pipe_dir.get_processed_step_path(current_step).joinpath("protectedseas_tiles.mbtiles")


# Download the protected seas file && unzip it
writeReadGCP(
    credentials=mysettings.GCS_KEYFILE_JSON,
    bucket_name=mysettings.GCS_BUCKET,
    blob_name="ProtectedSeas/ProtectedSeas_20240716.geojson",
    file=input_file,
    operation="r",
)

# Load the data
protectedseas_layer = gpd.read_file(input_file)

In [38]:
# simplify the geometries
Mapshaper(16).input([input_file.as_posix()]).clean(
    allow_overlaps=True, rewind=True
).simplify("dp 10% keep-shapes planar").clean(allow_overlaps=True).output(
    input_file.as_posix(),
    force=True,
    format="geojson",
).execute()

Allocating 16 GB of heap memory
[clean] Retained 6,741 of 6,741 features
[simplify] Repaired 5,240 intersections; 629 intersections could not be repaired
[clean] Retained 6,741 of 6,741 features
[o] Wrote /home/mambauser/data/protectedseas/processed/preprocess/ProtectedSeas_20240716.geojson


CompletedProcess(args='mapshaper-xl 16gb -i /home/mambauser/data/protectedseas/processed/preprocess/ProtectedSeas_20240716.geojson  -clean allow-overlaps rewind -simplify dp 10% keep-shapes planar -clean allow-overlaps -o /home/mambauser/data/protectedseas/processed/preprocess/ProtectedSeas_20240716.geojson force format=geojson', returncode=0)

In [39]:
mbtileGeneration(input_file, output_file)

PosixPath('/home/mambauser/data/protectedseas/processed/tiles/protectedseas_tiles.mbtiles')

In [41]:
uploadToMapbox(
    output_file,
    "protected_seas",
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

rosetta error: failed to open elf at /lib64/ld-linux-x86-64.so.2
 Trace/breakpoint trap


CalledProcessError: Command 'aws s3 cp /home/mambauser/data/protectedseas/processed/tiles/protectedseas_tiles.mbtiles s3://tilestream-tilesets-production/71/_pending/lxene85xrno1vgk2b6c6350mc/skytruth --region us-east-1' returned non-zero exit status 133.

### Habitat layers

In [ ]:
# TODO: Add all the habitats layer processing
habitat = mysettings.DATA_DIR.joinpath("habitat_intermediate", "layer").resolve()

#### Warm water corals

In [ ]:
warm_water_corals = habitat.joinpath("warm_water_corals.mbtiles").resolve()

In [ ]:
uploadToMapbox(
    warm_water_corals,
    "warm_water_corals",
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

### Contextual layers

In [ ]:
# Todo: we need to add the contextual layers upload here